In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense , merge
from keras.layers.merge import dot, Concatenate
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

import tensorflow as tf

def MF_layer(n_movies,n_users,n_latent_factors):
  user_input=Input(shape=(1,),name='user_input',dtype='int64')
  movie_input=Input(shape=(1,),name='movie_input',dtype='int64')

  user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
  movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)

  user_vec = Flatten(name='FlattenUsers')(user_embedding)
  movie_vec = Flatten(name='FlattenMovies')(movie_embedding)

  sim = dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)

  prediction = Dense(1, activation='linear', name = 'prediction')(sim)
  model = keras.models.Model([user_input, movie_input],prediction)

  return model

def MLP_layer(n_movies,n_users,n_latent_factors, layers = []):
  num_layer = len(layers)
  user_input=Input(shape=(1,),name='user_input',dtype='int64')
  movie_input=Input(shape=(1,),name='movie_input',dtype='int64')

  user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
  movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)

  user_vec = Flatten(name='FlattenUsers')(user_embedding)
  movie_vec= Flatten(name='FlattenMovies')(movie_embedding)

  sim = keras.layers.Concatenate()([user_vec, movie_vec])

  for idx in range(1, num_layer):
    layer = Dense(layers[idx], activation='relu', name = 'layer%d' %idx)
    sim = layer(sim)
  prediction = Dense(1, activation='relu', name = 'prediction')(sim)

  model = keras.models.Model([user_input, movie_input],prediction)

  return model

def NMF(n_movies,n_users,n_latent_factors,layers = [], NMF = False):
  num_layer = len(layers)
  user_input=Input(shape=(1,),name='user_input',dtype='int64')
  movie_input=Input(shape=(1,),name='movie_input',dtype='int64')

  MF_user_embedding=Embedding(n_users,n_latent_factors,name='MF_user_embedding')(user_input)
  MF_movie_embedding=Embedding(n_movies,n_latent_factors,name='MF_movie_embedding')(movie_input)

  MLP_user_embedding=Embedding(n_users,n_latent_factors,name='MLP_user_embedding')(user_input)
  MLP_movie_embedding=Embedding(n_movies,n_latent_factors,name='MLP_movie_embedding')(movie_input)

  MF_user_vec = Flatten(name='MF_FlattenUsers')(MF_user_embedding)
  MF_movie_vec= Flatten(name='MF_FlattenMovies')(MF_movie_embedding)
  MF_vector=dot([MF_user_vec,MF_movie_vec],name='Simalarity-Dot-Product',axes=1)

  MLP_user_vec = Flatten(name='MLP_FlattenUsers')(MLP_user_embedding)
  MLP_movie_vec= Flatten(name='MLP_FlattenMovies')(MLP_movie_embedding)
  MLP_vector = keras.layers.Concatenate()([MLP_user_vec, MLP_movie_vec])

  for idx in range(0, num_layer):
    layer = Dense(layers[idx], activation='relu', name = 'layer%d' %idx)
    MLP_vector = layer(MLP_vector)

  predict_vector = keras.layers.Concatenate()([MF_vector, MLP_vector])

  if NMF == True: 
    NMF_layer = Dense(200, activation='relu', name = 'NMF_1_layer')(predict_vector)
    NMF_layer = Dense(100, activation='relu', name = 'NMF_2_layer')(NMF_layer)
    prediction = Dense(1, activation='relu', name = 'prediction')(NMF_layer)

  prediction = Dense(1, activation='relu', name = 'prediction')(predict_vector)

  model = keras.models.Model([user_input, movie_input],prediction)

  return model

  